In [1]:
import scipy
import scipy.io
import matplotlib.pyplot as plt
import numpy as np
import random
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.neighbors import KNeighborsClassifier

In [2]:
data = np.load("all_features_4sec_epoch.npy")

# data = data[:, :, :, 2:6]

labels_ids = [[1,2,3,0,2,0,0,1,0,1,2,1,1,1,2,3,2,2,3,3,0,3,0,3],
        [2,1,3,0,0,2,0,2,3,3,2,3,2,0,1,1,2,1,0,3,0,1,3,1],
        [1,2,2,1,3,3,3,1,1,2,1,0,2,3,3,0,2,3,0,0,2,0,1,0]]
print(len(labels_ids))
labels = np.zeros(data.shape[:-3])
for session in range(data.shape[0]):
    for video in range(data.shape[2]):
        labels[session,:,video] = labels_ids[session][video]

print(labels.shape) # (3, 15, 24, 8)
print(data.shape) # (3, 15, 24, 8, 4, 62, 10)
print(np.logical_or((labels == 0), (labels == 2)).shape) # (3, 15, 24, 8)


3
(3, 15, 24, 10)
(3, 15, 24, 10, 4, 62, 10)
(3, 15, 24, 10)


In [3]:
def train_test(features, labels, test_subjects=[0,1,2]):
    test_data = features[:, test_subjects]
    test_labels = labels[:, test_subjects]

    train_data = np.delete(features, test_subjects, axis=1)
    train_labels = np.delete(labels, test_subjects, axis=1)

    return train_data, train_labels, test_data, test_labels

In [4]:

accs = []
for i in range(15):
    train_data, train_labels, test_data, test_labels = train_test(data, labels, test_subjects=[i])

    train_mask = np.logical_or((train_labels == 0), (train_labels == 2))
    test_mask = np.logical_or((test_labels == 0), (test_labels == 2))

    train_labels = train_labels[train_mask]
    train_data = train_data[train_mask]

    test_labels = test_labels[test_mask]
    test_data = test_data[test_mask]

    # test_data = np.reshape(test_data, (test_data.shape[0], -1))
    # train_data = np.reshape(train_data, (train_data.shape[0], -1))

    train_labels[train_labels == 0] = 0
    test_labels[test_labels == 0] = 0

    test_labels[test_labels == 2] = 1
    train_labels[train_labels == 2] = 1

    # print(train_data.shape)
    # print(train_labels.shape)

    # print(test_data.shape)
    # print(test_labels.shape)
    # knn = KNeighborsClassifier(n_neighbors=100)
    # knn.fit(train_data,train_labels)
    # predicted_data = knn.predict(test_data)
    # acc = accuracy_score(test_labels,predicted_data)
    # print(f"i {i}, acc {acc}")
    # accs.append(acc)
print(np.mean(accs))
# normalize data
# train_data = (train_data - np.mean(train_data, axis=0)) / np.std(train_data, axis=0)
# test_data = (test_data - np.mean(train_data, axis=0)) / np.std(train_data, axis=0)



nan


/usr/lib/python3.12/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/lib/python3.12/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [5]:
train_data, train_labels, test_data, test_labels = train_test(data, labels, test_subjects=[i])

train_mask = np.logical_or((train_labels == 0), (train_labels == 2))
test_mask = np.logical_or((test_labels == 0), (test_labels == 2))

train_labels = train_labels[train_mask]
train_data = train_data[train_mask]

test_labels = test_labels[test_mask]
test_data = test_data[test_mask]

# test_data = np.reshape(test_data, (sveska.ipynb.shape[0], -1))

train_labels[train_labels == 0] = 0
test_labels[test_labels == 0] = 0

test_labels[test_labels == 2] = 1
train_labels[train_labels == 2] = 1

In [6]:
# KORELACIONA ANALIZA SA LABELAMA
# df1 = pd.DataFrame(train_data)
# df2 = pd.DataFrame(test_data)
# df = pd.concat([df1, df2])
# df1['label'] = np.concatenate([train_labels])
# df2['label'] = np.concatenate([test_labels])

# df['label'] = np.concatenate((train_labels, test_labels))
# # save df to csv
# df.to_csv('data.csv', index=False)

# df2.to_csv('test_data.csv', index=False)
# df1.to_csv('train_data.csv', index=False)

# df
# corr_analysis=abs(df.corr()['label']).sort_values(ascending=False)
# corr_analysis = corr_analysis[corr_analysis.index!='label']
# corr_analysis


In [7]:
print(f"Shapes: {train_data.shape}, {train_labels.shape}, {test_data.shape}, {test_labels.shape}")

Shapes: (5040, 4, 62, 10), (5040,), (360, 4, 62, 10), (360,)


In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F

print(torch.cuda.is_available())
# clear cuda cache
torch.cuda.empty_cache()

True


In [53]:
class EEGEmotionClassifier(nn.Module):
    def __init__(self, num_channels=4, num_features_1=62, num_features_2=10, conv1_out_channels=16, conv2_out_channels=32, conv3_out_channels=64, fc1_out_features=128, fc2_out_features=64):
        super(EEGEmotionClassifier, self).__init__()
        
        # Convolutional layers
        self.conv1 = nn.Conv2d(in_channels=num_channels, out_channels=conv1_out_channels, kernel_size=(3, 3), stride=1, padding=1, dtype=torch.float32)
        self.conv2 = nn.Conv2d(in_channels=conv1_out_channels, out_channels=conv2_out_channels, kernel_size=(3, 3), stride=1, padding=1, dtype=torch.float32)
        self.conv3 = nn.Conv2d(in_channels=conv2_out_channels, out_channels=conv3_out_channels, kernel_size=(3, 3), stride=1, padding=1, dtype=torch.float32)
        
        # Pooling layer
        self.pool = nn.MaxPool2d(kernel_size=(2, 2), stride=2, padding=0)
        
        # Fully connected layers
        self.fc1 = nn.Linear(1920, fc1_out_features)
        self.fc2 = nn.Linear(fc1_out_features, fc2_out_features)
        self.fc3 = nn.Linear(fc2_out_features, 1)  # Output layer for binary classification
        
    def forward(self, x):
        # Convolutional layers with ReLU and pooling
        x = (F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))

        # Flatten the tensor
        x = x.view(-1, self.num_flat_features(x))
        
        # Fully connected layers with ReLU
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        
        # Output layer
        x = F.sigmoid(self.fc3(x))
        
        return x
    
    def num_flat_features(self, x):
        size = x.size()[1:]  # All dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


In [54]:
data = np.load("all_features_4sec_epoch.npy")

# data = data[:, :, :, 2:6]

labels_ids = [[1,2,3,0,2,0,0,1,0,1,2,1,1,1,2,3,2,2,3,3,0,3,0,3],
        [2,1,3,0,0,2,0,2,3,3,2,3,2,0,1,1,2,1,0,3,0,1,3,1],
        [1,2,2,1,3,3,3,1,1,2,1,0,2,3,3,0,2,3,0,0,2,0,1,0]]
print(len(labels_ids))
labels = np.zeros(data.shape[:-3])
for session in range(data.shape[0]):
    for video in range(data.shape[2]):
        labels[session,:,video] = labels_ids[session][video]

print(labels.shape) # (3, 15, 24, 8)
print(data.shape) # (3, 15, 24, 8, 4, 62, 10)
print(np.logical_or((labels == 0), (labels == 2)).shape) # (3, 15, 24, 8)

data = data.astype(np.float32)

train_data, train_labels, test_data, test_labels = train_test(data, labels, test_subjects=[i])

train_mask = np.logical_or((train_labels == 0), (train_labels == 2))
test_mask = np.logical_or((test_labels == 0), (test_labels == 2))

train_labels = train_labels[train_mask]
train_data = train_data[train_mask]

test_labels = test_labels[test_mask]
test_data = test_data[test_mask]


train_labels[train_labels == 0] = 0
test_labels[test_labels == 0] = 0

test_labels[test_labels == 2] = 1
train_labels[train_labels == 2] = 1

from torch.optim.lr_scheduler import CosineAnnealingLR

hidden_size = 500
net = EEGEmotionClassifier(num_channels=4, num_features_1=62, num_features_2=10, conv1_out_channels=16, conv2_out_channels=32, conv3_out_channels=64, fc1_out_features=128, fc2_out_features=64).to('cuda')
optimizer = torch.optim.AdamW(net.parameters(), lr=0.0001)
# scheduler = CosineAnnealingLR(optimizer, T_max=10)


loss_fn = nn.BCELoss()

# Normalize train and test data



train_data = torch.tensor(train_data, dtype=torch.float32).to('cuda')
train_labels = torch.tensor(train_labels, dtype=torch.long).to('cuda')
test_data = torch.tensor(test_data, dtype=torch.float32)
test_labels = torch.tensor(test_labels, dtype=torch.long)

from torch.utils.data import DataLoader, TensorDataset

# Create TensorDatasets for training and testing data
train_dataset = TensorDataset(train_data, train_labels)
test_dataset = TensorDataset(test_data, test_labels)

# Create DataLoaders for training and testing data
train_loader = DataLoader(train_dataset, batch_size=500, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1000, shuffle=False)


# convert labels to one-hot encoding
# train_labels = F.one_hot(train_labels, num_classes=4).float()
# test_labels = F.one_hot(test_labels, num_classes=4).float()



3
(3, 15, 24, 10)
(3, 15, 24, 10, 4, 62, 10)
(3, 15, 24, 10)


In [55]:
net.train()
def init_weights(m):
    if isinstance(m, nn.Linear):
        torch.nn.init.xavier_uniform(m.weight)
        m.bias.data.fill_(0.01)
net.apply(init_weights)
model = net
best_val_acc = 0
for epoch in range(100000):
    net.train()
    # clear cuda cache
    torch.cuda.empty_cache()
    for i, (data, labels) in enumerate(train_loader):
        optimizer.zero_grad()
        out = net(data)[..., 0]

        loss = loss_fn(out.float(), labels.float())
        loss.backward()
        optimizer.step()
    # scheduler.step()


    if epoch % 10 == 0:
        out = []
        
        for i, (data, labels) in enumerate(train_loader):
            out.append(net(data)[..., 0].cpu())
        out = torch.cat(out)

        acc = accuracy_score(train_labels.cpu(), out > 0.5)
        # acc = accuracy_score(torch.argmax(train_labels, dim=1).cpu(), torch.argmax(out, dim=1).cpu())
        # test the model
        with torch.no_grad():
            outs = []
            for i, (data, labels) in enumerate(test_loader):
                data = data.to('cuda')
                out = net(data)[..., 0]
                loss = loss_fn(out.cpu(), labels.float())
                outs.append(out.cpu())
            out = torch.cat(outs)
            test_acc = accuracy_score(test_labels.cpu(), out > 0.5)
        print(f'Epoch {epoch}, loss {loss.item()} train_accuracy {acc} test_accuracy {test_acc}')

        if test_acc > best_val_acc:
            best_val_acc = test_acc
            print(f'New best val accuracy: {best_val_acc} at epoch {epoch}')    




/tmp/ipykernel_15762/467034894.py:4: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  torch.nn.init.xavier_uniform(m.weight)


Epoch 0, loss 0.6961286067962646 train_accuracy 0.5005952380952381 test_accuracy 0.5
New best val accuracy: 0.5 at epoch 0
Epoch 10, loss 0.7102961540222168 train_accuracy 0.5 test_accuracy 0.5
Epoch 20, loss 0.6924713253974915 train_accuracy 0.5027777777777778 test_accuracy 0.5055555555555555
New best val accuracy: 0.5055555555555555 at epoch 20
Epoch 30, loss 0.6900306344032288 train_accuracy 0.4998015873015873 test_accuracy 0.5305555555555556
New best val accuracy: 0.5305555555555556 at epoch 30
Epoch 40, loss 0.6875377893447876 train_accuracy 0.5077380952380952 test_accuracy 0.5694444444444444
New best val accuracy: 0.5694444444444444 at epoch 40
Epoch 50, loss 0.6836801171302795 train_accuracy 0.5027777777777778 test_accuracy 0.5722222222222222
New best val accuracy: 0.5722222222222222 at epoch 50
Epoch 60, loss 0.6798761487007141 train_accuracy 0.49563492063492065 test_accuracy 0.6166666666666667
New best val accuracy: 0.6166666666666667 at epoch 60
Epoch 70, loss 0.6796131730079

KeyboardInterrupt: 

In [28]:
# # generate confusion matrix
# net.eval()
# with torch.no_grad():
#     out = net(test_data.to('cuda'))
#     predicted_labels = torch.argmax(out, dim=1)
#     test_non_one_hot = torch.argmax(test_labels, dim=1)
#     test_acc = accuracy_score(test_non_one_hot, predicted_labels.cpu())
#     cm = confusion_matrix(test_non_one_hot, predicted_labels.cpu())
#     print(f"Test accuracy: {test_acc}")
#     print(cm)
#     disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['0','1','2','3'])
#     disp.plot()
#     plt.show()

In [ ]:
def create_model(trial):
    hidden_size = trial.suggest_int('hidden_size', 100, 2000)
    dropout_rate = trial.suggest_float('dropout_rate', 0.2, 0.7)
    lr = trial.suggest_float('lr', 1e-5, 1e-1, log=True)
    weight_decay = trial.suggest_float('weight_decay', 1e-8, 1e-3, log=True)
    
    # model = nn.Sequential(
    #     nn.Linear(10*62*4, hidden_size*2),
    #     nn.BatchNorm1d(hidden_size*2),
    #     nn.LeakyReLU(),
    #     nn.Dropout(dropout_rate),
    #     nn.Linear(hidden_size*2, hidden_size),
    #     nn.BatchNorm1d(hidden_size),
    #     nn.LeakyReLU(),
    #     nn.Dropout(dropout_rate),
    #     nn.Linear(hidden_size, hidden_size//2),
    #     nn.BatchNorm1d(hidden_size // 2),
    #     nn.LeakyReLU(),
    #     nn.Dropout(dropout_rate),
    #     nn.Linear(hidden_size//2, 1),
    #     nn.Sigmoid()
    # ).to('cuda')

    model = nn.Sequential(
        nn.Linear(10*62*4, hidden_size),
        nn.BatchNorm1d(hidden_size),
        nn.ReLU(),
        nn.Dropout(dropout_rate),
        nn.Linear(hidden_size, 1),
        nn.Sigmoid()
    ).to('cuda')
    model.apply(init_weights)


    optimizer = torch.optim.AdamW(net.parameters(), lr=lr, weight_decay=weight_decay)

    
    # optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)
    return model, optimizer

In [ ]:
def objective(trial):
    model, optimizer = create_model(trial)
    loss_fn = nn.BCELoss()
    best_val_acc = 0.0
    for epoch in range(500):    
        model.train()
        torch.cuda.empty_cache()
        
        # Training loop with DataLoader
        # for batch_data, batch_labels in train_loader:
        #     batch_data, batch_labels = batch_data.to('cuda'), batch_labels.to('cuda')
        #     optimizer.zero_grad()
        #     out = model(batch_data)
        #     loss = loss_fn(out[..., 0], batch_labels.float())
        #     optimizer.step()
            
        #     loss.backward()

        optimizer.zero_grad()
        out = model(train_data)
        # print(f"out: {out.shape}, train_labels: {train_labels.shape}, train_data: {train_data.shape}")
        loss = loss_fn(out[..., 0], train_labels.float())
        
        loss.backward()
        optimizer.step()

        
        # Validation step
        model.eval()
        with torch.no_grad():
            # Calculate training accuracy
            # all_train_out = []
            # for batch_data, _ in train_loader:
            #     batch_data = batch_data.to('cuda')
            #     out = model(batch_data)
            #     all_train_out.append(out[..., 0].cpu())
            # all_train_out = torch.cat(all_train_out)
            # train_acc = accuracy_score(train_labels.cpu(), all_train_out > 0.5)
            
            # # Calculate validation accuracy
            # all_test_out = []
            # for batch_data, _ in test_loader:
            #     batch_data = batch_data.to('cuda')
            #     out = net(batch_data)
            #     all_test_out.append(out[..., 0].cpu())
            # all_test_out = torch.cat(all_test_out)
            # val_acc = accuracy_score(test_labels, all_test_out > 0.5)
            
            out = out[..., 0].detach().cpu() > 0.5
            out = out
            train_acc = accuracy_score(train_labels.cpu(), out)
            # acc = accuracy_score(torch.argmax(train_labels, dim=1).cpu(), torch.argmax(out, dim=1).cpu())
            # test the model
            with torch.no_grad():
                out = net(test_data.to('cuda'))
                # predicted_labels = torch.argmax(out, dim=1)
                # test_non_one_hot = torch.argmax(test_labels, dim=1)
                val_acc = accuracy_score(test_labels, out[..., 0].cpu() > 0.5)


        if val_acc > best_val_acc:
            best_val_acc = val_acc
            print(f'New best val accuracy: {best_val_acc} at epoch {epoch}')
        
        if epoch % 50 == 0:
            print(f'Epoch {epoch}, loss {loss.item()}, train_acc {train_acc}, val_accuracy {val_acc}')
    
    return best_val_acc


In [ ]:
import optuna
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50, n_jobs=1)


print(f'Best hyperparameters: {study.best_params}')


[I 2024-06-03 17:52:11,360] A new study created in memory with name: no-name-3cebeba2-b258-4cd1-a2e6-d30e84b1b15a
/tmp/ipykernel_13991/1746426871.py:4: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  torch.nn.init.xavier_uniform(m.weight)


New best val accuracy: 0.5222222222222223 at epoch 0
Epoch 0, loss 0.8216129541397095, train_acc 0.5001984126984127, val_accuracy 0.5222222222222223
New best val accuracy: 0.5388888888888889 at epoch 1
New best val accuracy: 0.5444444444444444 at epoch 7
New best val accuracy: 0.55 at epoch 22
New best val accuracy: 0.5583333333333333 at epoch 40
New best val accuracy: 0.5611111111111111 at epoch 46
Epoch 50, loss 0.8271164894104004, train_acc 0.4996031746031746, val_accuracy 0.5333333333333333
New best val accuracy: 0.5638888888888889 at epoch 66
Epoch 100, loss 0.8359272480010986, train_acc 0.49404761904761907, val_accuracy 0.5361111111111111
Epoch 150, loss 0.835940420627594, train_acc 0.5007936507936508, val_accuracy 0.5333333333333333
Epoch 200, loss 0.8274425864219666, train_acc 0.5063492063492063, val_accuracy 0.5361111111111111


[W 2024-06-03 17:52:21,781] Trial 0 failed with parameters: {'hidden_size': 1344, 'dropout_rate': 0.40195477741862107, 'lr': 0.017016176881559823, 'weight_decay': 4.692128365995673e-06} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/home/drakula/.local/lib/python3.12/site-packages/optuna/study/_optimize.py", line 196, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "/tmp/ipykernel_13991/4244170460.py", line 49, in objective
    out = out[..., 0].detach().cpu() > 0.5
          ^^^^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt
[W 2024-06-03 17:52:21,782] Trial 0 failed with value None.


KeyboardInterrupt: 